In [1]:
import csv
import pandas as pd
import shap
import sklearn
import numpy as np
import matplotlib
import interpret.glassbox
import xgboost
import matplotlib.pylab as pl
import shap
import xgboost
%matplotlib inline 
matplotlib.use('tkagg')
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl
import os
from os import listdir
from os.path import isfile, join
from collections import Counter
import time
from scipy.stats import randint, uniform
from sklearn.utils.fixes import loguniform
import seaborn as sns
import missingno as msno
import pandas_profiling
from sklearn.metrics import roc_auc_score
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from hyperopt.pyll.base import scope
import pyspark
data = pd.read_csv('cohort.csv')
# data = pd.read_csv('cohortpetit.csv')
# data2 = pd.read_csv('cohort100.csv')

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [2]:
# Define functions

# Hyperparameters tuning

from hyperopt import STATUS_OK, fmin, hp, tpe, SparkTrials

# Some constants

SEED = 1234
VALID_SIZE = 0.2
TARGET = 'outcome'

def c_statistic_harrell(pred, labels):
    total = 0
    matches = 0
    for i in range(len(labels)):
        for j in range(len(labels)):
            if int(labels[j]) > 0 and abs(int(labels[i])) > int(labels[j]):
                total += 1
                if pred[j] > pred[i]:
                    matches += 1
    return matches/total

#-----------------------XGBoost--------------------------#

def score_xgb(params):
    print("Training with params: ")
    print(params)
    
    n_folds = 5
    val_scores = []
    skf = KFold(n_splits = n_folds, shuffle = False)
    num_boost_round=2000
    #k-fold CV
    for train_index, val_index in skf.split(X_train): 
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        dtrain = xgboost.DMatrix(X_tr, label=y_tr)
        dval = xgboost.DMatrix(X_val, label=y_val)

        watchlist = [(dval, 'eval'), (dtrain, 'train')]
        gbm_model = xgboost.train(params, dtrain, num_boost_round,
                              evals=watchlist,
                              verbose_eval=500)
        
        predictions = gbm_model.predict(dval,
                                    ntree_limit=gbm_model.best_iteration + 1)
        val_scores.append(c_statistic_harrell(predictions, list(y_val)))
    
    score = np.mean(val_scores) #Objective: maximize mean 5-fold CV C-index 
    
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}

def optimize_xgb(score, 
             random_state=SEED):
    """
    This is the optimization function that given a space of 
    hyperparameters and a scoring function, finds the best hyperparameters.
    """
    # XGB param ranges obtained from Barnwal A, Cho H, Hocking T (2020). Survival regression with accelerated failure time model in XGBoost: 
    # With some excpetions as I observed that the optimal range is less wider that what they suggest
    space = {
        'eta':                         hp.loguniform('eta', np.log(0.001), np.log(1)),
        'max_depth':                   scope.int(hp.quniform('max_depth', 2,10,1)),
        'min_child_weight':            hp.loguniform('min_child_weight', np.log(0.001), np.log(10)),
        'reg_alpha':                   hp.loguniform('reg_alpha', np.log(0.001), np.log(10)),
        'reg_lambda':                  hp.loguniform('reg_lambda', np.log(0.001), np.log(10)),
        'subsample':                   hp.uniform('subsample', 0.75, 1),
        "objective": "survival:cox",
        "predictor": "gpu_predictor"
    }
    
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                max_evals=100)
    return best

In [3]:
y = data['time']
y_event = data['event']
y = y.head(1000)
y_event = y_event.head(1000)
X = data.drop(['time'], axis = 1)
X = X.drop(['event'], axis = 1)
X=X.head(1000)

In [ ]:
y

In [ ]:
y_event

In [4]:
# feature_names = [i for i in data.columns if data[i].dtype in [np.int64, np.int64]]
feature_names = [i for i in X.columns ]
X = X[feature_names]
X = X.head(1000)
# my_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)
# model_ebm = interpret.glassbox.ExplainableBoostingRegressor()
# model_ebm.fit(X, y)
##########
# X100 = data2[feature_names]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [ ]:
feature_names

In [ ]:
print("---- Running XGBoost ----")
start=time.time()
xgb_best_hyperparams = optimize_xgb(score_xgb)
end=time.time()
print("Done: took", (end-start)/60, "minutes")
print("---- Results for XGBoost ----")
print("The best hyperparameters are: ", "\n")
print(xgb_best_hyperparams)

In [ ]:
xgb_best_hyperparams['max_depth']=round(xgb_best_hyperparams['max_depth'])
xgb_best_hyperparams['objective']="survival:cox"
xgb_best_hyperparams['predictor']="gpu_predictor"
print(xgb_best_hyperparams)

In [ ]:
X.columns

In [6]:
num_boost_round=2000
dtrain = xgboost.DMatrix(X_train, label=y_train)
dval = xgboost.DMatrix(X_test, label=y_test)

watchlist = [(dval, 'eval'), (dtrain, 'train')]
xgb_model = xgboost.train(xgb_best_hyperparams, dtrain, num_boost_round,
                      evals=watchlist,
                      verbose_eval=500)

predictions = xgb_model.predict(dval,
                            ntree_limit=xgb_model.best_iteration + 1)

xgb_test_score = c_statistic_harrell(predictions, list(y_test))
print("XGB Done:", xgb_test_score)



pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[0]	eval-cox-nloglik:5.18859	train-cox-nloglik:6.50720
[500]	eval-cox-nloglik:5.18649	train-cox-nloglik:6.48958
[1000]	eval-cox-nloglik:5.19037	train-cox-nloglik:6.48091
[1500]	eval-cox-nloglik:5.19416	train-cox-nloglik:6.47528
[1999]	eval-cox-nloglik:5.19735	train-cox-nloglik:6.47083
XGB Done: 0.660853316723762


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [7]:
params = {
    'eta': 0.0012482292166029548,
    "max_depth": 3,
    'min_child_weight': 2.1899878853989163,
    'reg_alpha': 0.16737772985552463,
    'reg_lambda': 0.21803902714013768,
    "objective": "survival:cox",
    "subsample":  0.8874909590869486,
    "predictor": "gpu_predictor"
}
# model = xgboost.train(params, xgb_full, 10000, evals = [(xgb_full, "test")], verbose_eval=1000)
full_xy = X.copy()
y_labels = y.copy()
full_xy['y'] = y_labels
X_features = X.columns

In [8]:
full_xy = X.copy()
y_labels = y.copy()
full_xy['y'] = y_labels
X_features = X.columns
myHazRatios = [[] for i in range(len(X.columns))]
myIndexes = [i for i in range(len(X.columns))]
# print(myHazRatios_all, myIndexes_all)
myBinCols = [col for col in X.columns if all(X[col].value_counts().index.isin([0, 1]))]
myContCols = [col for col in X.columns if not all(X[col].value_counts().index.isin([0, 1]))]

print("My list of binary columns are :\n", myBinCols)

print("\nMy list of cont. columns are :\n", myContCols)

My list of binary columns are :
 ['men', 'smoker', 'p.dm', 'htn_med', 'c10']

My list of cont. columns are :
 ['age', 'pas', 'ct', 'chdl', 'cldl', 'tg', 'charlson', 'ckd.epi', 'hematocrit']


In [9]:
for i in range(1, 1001):
#   sample with replacement:
    X_train = full_xy.sample(n = full_xy.shape[0], random_state = i, replace = True) #sample w/ replacement num rows
    y_train = X_train['y'] #y train are X_trains y column
    X_train = X_train.drop(columns = ['y']) #X_train then needs to drop the y column
    X_test = full_xy.drop(X_train.index) #test features are the full - X_train
    y_test = X_test['y'] #y test are X_tests y column
    X_test = X_test.drop(columns = ['y']) #X_test then needs to drop they column
    xgb_train = xgboost.DMatrix(X_train, label=y_train)
    xgb_test = xgboost.DMatrix(X_test, label=y_test)
    print('iteration', i)
    model_train = xgboost.train(params, xgb_train, 2000, evals = [(xgb_test, "test")], verbose_eval=False)
    
#   get the SHAP vals and a HR from these based on our model
    shap_values_full = shap.TreeExplainer(model_train).shap_values(X)
    
#   build a list of HazRatios indexing by j columns
    for j in range(len(myIndexes)):
        curSHAP = shap_values_full[:, j]
        curCol = X.columns[j]
        # if continuous, split by mean
        if (curCol in myContCols):
            myMean = X[curCol].mean()
            myHazRatios[j].append(np.mean(np.exp(curSHAP[X[curCol] >= myMean]))
                                      /np.mean(np.exp(curSHAP[X[curCol] < myMean])))
        # else, split by 1 or 0
        else:
            myHazRatios[j].append(np.mean(np.exp(curSHAP[X[curCol] == 1]))
                                    /np.mean(np.exp(curSHAP[X[curCol] == 0])))



pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


iteration 2


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


iteration 3


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


iteration 4


In [10]:
# myHazRatios 
# indexed by column index
X_features = X.columns
mySummaries = []
for i in range(len(myHazRatios)):
    myHazRatios[i].sort()
    mySummaries.append([np.mean(myHazRatios[i])])
print(mySummaries)
print(X.columns)

[[1.1401471], [1.004879], [1.2034696], [1.0091059], [0.97980917], [1.0057719], [0.9798243], [0.9776175], [1.0245229], [0.9793414], [1.0311009], [0.9471054], [0.99844146], [1.0351961]]
Index(['age', 'men', 'smoker', 'p.dm', 'pas', 'ct', 'chdl', 'cldl', 'tg',
       'htn_med', 'charlson', 'ckd.epi', 'c10', 'hematocrit'],
      dtype='object')


In [ ]:
xgb_full = xgboost.DMatrix(X, label=y)

# create a train/test split
xgb_train = xgboost.DMatrix(X_train, label=y_train)
xgb_test = xgboost.DMatrix(X_test, label=y_test)
watchlist = [(xgb_train, 'train'), (xgb_test, 'test')]
# model_train = xgboost.train(params, xgb_train, 20000, evals = [(xgb_test, "test")], verbose_eval=1000)

In [ ]:
model = xgboost.train(params, xgb_train, 20000, evals = watchlist, verbose_eval=1000)

In [ ]:
def c_statistic_harrell(pred, labels):
    total = 0
    matches = 0
    for i in range(len(labels)):
        for j in range(len(labels)):
            if int(labels[j]) > 0 and abs(int(labels[i])) > int(labels[j]):
                total += 1
                if pred[j] > pred[i]:
                    matches += 1
    return matches/total

# see how well we can order people by survival, 1 = perfect ordering
c_statistic_harrell(model.predict(xgb_full, ntree_limit=15000), list(y))

In [ ]:
shap_values = shap.TreeExplainer(model).shap_values(X)

In [ ]:
matplotlib.use('tkagg',force=True)
from matplotlib import pyplot as plt
matplotlib.get_backend()
%matplotlib inline

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
shap.summary_plot(shap_values, X, plot_type='bar')

In [ ]:
params2 = {
    'eta': 0.001178734512359433,
    "max_depth": 3,
    'min_child_weight': 1.497139871118963,
    'reg_alpha': 0.14836099959046609,
    'reg_lambda': 0.729907543598251,
    "objective": "binary:logistic",
    "subsample": 0.8506093620966385,
    "predictor": "gpu_predictor"
}
X_train, X_test, y_train, y_test = train_test_split(X, y_event, test_size=0.2, random_state=20)

In [ ]:
xgb_full = xgboost.DMatrix(X, label=y_event)

# create a train/test split
xgb_train = xgboost.DMatrix(X_train, label=y_train)
xgb_test = xgboost.DMatrix(X_test, label=y_test)
watchlist = [(xgb_train, 'train'), (xgb_test, 'test')]

In [ ]:
model2 = xgboost.train(params2, xgb_train, 20000, evals = watchlist, verbose_eval=1000)

In [ ]:
c_statistic_harrell(model2.predict(xgb_full, ntree_limit=20000), list(y_event))

In [ ]:
y_event.mean()

In [ ]:
shap.initjs()
explainerModel = shap.TreeExplainer(model2)
shap_values_Model = explainerModel.shap_values(X)
def shap_plot(j):
    p = shap.force_plot(explainerModel.expected_value, shap_values_Model[j], X.iloc[[j]])
    return(p)
shap_plot(50)

In [ ]:
shap_plot(40)

In [ ]:
shap_plot(80)

In [ ]:
X.loc[X['age']==100]

In [ ]:
shap_plot(4619)

In [ ]:
X.loc[X['charlson']>=15]

In [ ]:
shap_plot(608)

In [ ]:
shap_plot(4300)

In [ ]:
# we pass "age" instead of an index because dependence_plot() will find it in X's column names for us
# glucosa was automatically chosen for coloring based on a potential interaction to check that
# the interaction is really in the model see SHAP interaction values below
shap.dependence_plot("charlson", shap_values, X)

In [ ]:
shap.dependence_plot("chdl", shap_values, X)

In [ ]:
shap.dependence_plot("men", shap_values, X)

In [ ]:
# takes a couple minutes since SHAP interaction values take a factor of 2 * # features
# more time than SHAP values to compute
shap_interaction_values = shap.TreeExplainer(model).shap_interaction_values(X)

In [ ]:
shap.summary_plot(shap_interaction_values, X)

In [ ]:
shap.dependence_plot(
    ("age", "ckd.epi"),
    shap_interaction_values, X
)

In [ ]:
shap.dependence_plot(
    ("age", "tg"),
    shap_interaction_values, X
)

In [ ]:
import numpy as np

In [ ]:
tmp = np.abs(shap_interaction_values).sum(0)
for i in range(tmp.shape[0]):
    tmp[i,i] = 0
inds = np.argsort(-tmp.sum(0))[:50]
tmp2 = tmp[inds,:][:,inds]
pl.figure(figsize=(12,12))
pl.imshow(tmp2)
pl.yticks(range(tmp2.shape[0]), X.columns[inds], rotation=50.4, horizontalalignment="right")
pl.xticks(range(tmp2.shape[0]), X.columns[inds], rotation=50.4, horizontalalignment="left")
pl.gca().xaxis.tick_top()
pl.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

shap_pca2 = PCA(n_components=2).fit_transform(shap_values[:,:-1])

In [ ]:
f = pl.figure(figsize=(5,5))
pl.scatter(shap_pca2[:,0], shap_pca2[:,1], c=np.sum(shap_values,axis=1), linewidth=0, alpha=0.5)
cb = pl.colorbar(label="Model output", aspect=40, orientation="horizontal")
cb.set_alpha(1)
cb.draw_all()
cb.outline.set_linewidth(0)
cb.ax.tick_params('x', length=0)
cb.ax.xaxis.set_label_position('top')
pl.gca().axis("off")
pl.show()